# Notebook 03: Transition-State Theory (TST)

## 1. Introduction: The Activated Complex

Collision theory is intuitive but hard to calculate (steric factors are empirical). **Transition-State Theory (TST)**, developed by Eyring, Evans, and Polanyi (1930s), takes a different approach. It focuses on the species at the top of the energy barrier: the **activated complex** or **transition state** ($C^\ddagger$).

### Key Assumptions
1.  **Quasi-Equilibrium**: Reactants A and B are in equilibrium with the activated complex $C^\ddagger$.
    $$ A + B \underset{k_{-1}}{\overset{k_1}{\rightleftharpoons}} C^\ddagger \xrightarrow{k^\ddagger} P $$
2.  **Motion along Reaction Coordinate**: The complex falls apart into products with a frequency $\nu^\ddagger$ (a "loose vibration").

### Learning Objectives
1.  Understand the statistical mechanical basis of the Eyring equation.
2.  Relate rate constants to thermodynamic parameters ($\Delta^\ddagger G$, $\Delta^\ddagger H$, $\Delta^\ddagger S$).
3.  Explain Kinetic Isotope Effects (KIE) and Quantum Tunneling.
4.  Describe the Kinetic Salt Effect for ionic reactions.
5.  Apply Linear Free Energy Relationships (Hammett Plots).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants
import ipywidgets as widgets
from IPython.display import display

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'figure.dpi': 120,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'DejaVu Sans'],
    'grid.alpha': 0.3
})

k_B = constants.Boltzmann
h = constants.Planck
R = constants.gas_constant
N_A = constants.Avogadro

print("Libraries loaded.")

## 2. The Eyring Equation

The concentration of the activated complex is given by statistical thermodynamics:
$$ K^\ddagger = \frac{[C^\ddagger]}{[A][B]} = \frac{q_{C^\ddagger}}{q_A q_B} e^{-E_0/RT} $$

**Crucial Step**: We factor out the vibrational mode corresponding to the reaction coordinate from the partition function $q_{C^\ddagger}$:
$$ q_{C^\ddagger} \approx \left(\frac{k_B T}{h \nu^\ddagger}\right) \bar{q}_{C^\ddagger} $$

The rate is $Rate = \nu^\ddagger [C^\ddagger]$. Substituting $[C^\ddagger]$:
$$ Rate = \nu^\ddagger \times \left( \frac{k_B T}{h \nu^\ddagger} \frac{\bar{q}_{C^\ddagger}}{q_A q_B} e^{-E_0/RT} \right) [A][B] $$

The frequency $\nu^\ddagger$ cancels out! This gives the **Eyring Equation**:
$$ k_r = \kappa \frac{k_B T}{h} \bar{K}^\ddagger $$
where $\kappa$ is the transmission coefficient (usually $\approx 1$).

In thermodynamic terms ($\Delta^\ddagger G^\circ = -RT \ln \bar{K}^\ddagger$):
$$ k_r = \kappa \frac{k_B T}{h} e^{-\Delta^\ddagger G^\circ / RT} $$

![Reaction Coordinate](images/reaction_coordinate_diagram.png)

In [ ]:
def calculate_eyring(T, dG_act_kJ, kappa):
    dG_act = dG_act_kJ * 1000  # J/mol
    
    # Pre-exponential factor (kT/h)
    # At 300 K, kT/h approx 6 x 10^12 s^-1 (vibrational frequency scale)
    pre_exp = (k_B * T) / h
    
    # Rate constant
    kr = kappa * pre_exp * np.exp(-dG_act / (R * T))
    
    print(f"Temperature: {T} K")
    print(f"Activation Gibbs Energy: {dG_act_kJ} kJ/mol")
    print(f"Universal Frequency (kT/h): {pre_exp:.2e} s^-1")
    print(f"Rate Constant kr: {kr:.2e} s^-1")

widgets.interact(calculate_eyring, 
                 T=widgets.FloatSlider(min=200, max=1000, step=10, value=298, description='T (K)'),
                 dG_act_kJ=widgets.FloatSlider(min=10, max=200, step=1, value=80, description='dG_act (kJ/mol)'),
                 kappa=widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=1.0, description='kappa'));

## 3. Thermodynamic Formulation

We can break down the Gibbs energy of activation into enthalpy and entropy terms:
$$ \Delta^\ddagger G^\circ = \Delta^\ddagger H^\circ - T\Delta^\ddagger S^\circ $$

Substituting this into the Eyring equation:
$$ k_r = \frac{k_B T}{h} e^{\Delta^\ddagger S^\circ / R} e^{-\Delta^\ddagger H^\circ / RT} $$

### Comparison with Arrhenius Equation
Recall $k = A e^{-E_a/RT}$.
-   **Enthalpy** ($\Delta^\ddagger H^\circ$) corresponds to **Activation Energy** ($E_a$).
    -   Solution: $E_a = \Delta^\ddagger H^\circ + RT$
-   **Entropy** ($\Delta^\ddagger S^\circ$) corresponds to the **Pre-exponential Factor** ($A$).
    -   $A = \frac{e k_B T}{h} e^{\Delta^\ddagger S^\circ / R}$

**Entropy of Activation ($\Delta^\ddagger S^\circ$)**:
-   $\Delta^\ddagger S^\circ < 0$: Transition state is **more ordered** (associative, e.g., dimerization). Corresponds to steric factor $P < 1$.
-   $\Delta^\ddagger S^\circ > 0$: Transition state is **disordered** (dissociative, e.g., bond breaking).

In [ ]:
class ActivationExplorer:
    """Explore relationships between activation parameters"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.dH_slider = widgets.FloatSlider(min=10, max=100, step=1, value=50, description='Δ‡H (kJ/mol)')
        self.dS_slider = widgets.FloatSlider(min=-200, max=50, step=5, value=-40, description='Δ‡S (J/K·mol)')
        self.T_range = np.linspace(200, 500, 100)
        
    def plot(self, dH_kJ, dS_J):
        dH = dH_kJ * 1000
        dS = dS_J
        T = self.T_range
        
        # Calculate k_Eyring
        k_eyring = (k_B * T / h) * np.exp(dS / R) * np.exp(-dH / (R * T))
        
        # Calculate Arrhenius parameters (at mean T)
        T_mean = 350
        Ea = dH + R * T_mean
        A = (np.e * k_B * T_mean / h) * np.exp(dS / R)
        k_arrhenius = A * np.exp(-Ea / (R * T))
        
        # Calculate dG
        dG = dH - T * dS
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Eyring vs Arrhenius
        ax1.semilogy(1000/T, k_eyring, 'b-', linewidth=2, label='Eyring (Exact)')
        ax1.semilogy(1000/T, k_arrhenius, 'r--', label=f'Arrhenius Fit (at {T_mean}K)')
        ax1.set_xlabel('1000 / T (K⁻¹)', fontsize=12)
        ax1.set_ylabel('Rate Constant k (s⁻¹)', fontsize=12)
        ax1.set_title('Eyring vs Arrhenius Plot', fontsize=14)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Free Energy Contributions
        ax2.plot(T, dG/1000, 'g-', linewidth=2, label='Δ‡G')
        ax2.plot(T, np.full_like(T, dH/1000), 'r--', label='Δ‡H (Enthalpy)')
        ax2.plot(T, -T*dS/1000, 'b--', label='-TΔ‡S (Entropy)')
        ax2.set_xlabel('Temperature (K)', fontsize=12)
        ax2.set_ylabel('Energy (kJ/mol)', fontsize=12)
        ax2.set_title('Thermodynamic Contributions to Δ‡G', fontsize=14)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n{'='*60}")
        print(f"ACTIVATION PARAMETERS ANALYSIS")
        print(f"{'='*60}")
        print(f"Enthalpy (Δ‡H):        {dH_kJ} kJ/mol")
        print(f"Entropy (Δ‡S):         {dS_J} J/(K·mol)")
        print(f"Gibbs Energy (300K):   {(dH - 300*dS)/1000:.2f} kJ/mol")
        print(f"\n🎯 Arrhenius Equivalent (at {T_mean}K):")
        print(f"   Ea ≈ {Ea/1000:.2f} kJ/mol")
        print(f"   A  ≈ {A:.2e} s⁻¹")
        print(f"{'='*60}")
        
    def setup_display(self):
        widgets.interact(self.plot, dH_kJ=self.dH_slider, dS_J=self.dS_slider)

print("\n🔥 Activation Parameter Explorer:")
explorer = ActivationExplorer()

### ❓ Concept Check

<details>
<summary><strong>Q: A reaction has a highly negative entropy of activation ($\Delta^\ddagger S \ll 0$). What does this suggest about the mechanism?</strong></summary>

It suggests an **associative mechanism** where two or more molecules come together to form a highly ordered transition state, losing translational and rotational freedom.
</details>

## 4. Kinetic Isotope Effect (KIE) and Tunneling

Replacing a hydrogen atom (H) with deuterium (D) often slows down a reaction. Why?

### Zero-Point Energy (ZPE)
Chemical bonds vibrate even at absolute zero. The energy is $E_0 = \frac{1}{2} h \nu$.
Since $\nu \propto 1/\sqrt{\mu}$, and $\mu_D > \mu_H$, the C-D bond has lower frequency and lower ZPE than C-H.

$$ ZPE_{C-D} < ZPE_{C-H} $$

This means the C-H bond starts from a higher energy "platform" than C-D, so it needs less activation energy to reach the top.

$$ k_H > k_D $$

The maximum primary KIE ($k_H/k_D$) at room temperature is about 7.

### Quantum Tunneling
Light particles like H (and especially $e^-$) can tunnel *through* the barrier instead of going over it. Tunneling is much harder for D (heavier).

**Evidence for Tunneling:**
1.  $k_H/k_D \gg 7$ (can be > 100!).
2.  Curved Arrhenius plots (concave up) at low temperatures.

In [ ]:
def calculate_kie(wavenumber_cm1, T):
    # Convert wavenumber to frequency
    c = constants.c * 100  # cm/s
    nu_H = wavenumber_cm1 * c
    
    # Reduced mass approximation: nu ~ 1/sqrt(mu)
    # nu_D = nu_H / sqrt(2)
    nu_D = nu_H / np.sqrt(2)
    
    # ZPE difference (assuming transition state difference is negligible)
    # delta_Ea = ZPE_H - ZPE_D
    delta_ZPE = 0.5 * h * (nu_H - nu_D)
    
    # KIE = exp(delta_ZPE / kT)
    kie = np.exp(delta_ZPE / (k_B * T))
    
    print(f"C-H Stretch: {wavenumber_cm1} cm^-1")
    print(f"Temperature: {T} K")
    print(f"ZPE Difference: {delta_ZPE / constants.elementary_charge:.3f} eV")
    print(f"Theoretical Max KIE (kH/kD): {kie:.2f}")
    
    if kie > 7:
        print("Note: Values significantly > 7 might imply tunneling!")

widgets.interact(calculate_kie, 
                 wavenumber_cm1=widgets.FloatSlider(min=1000, max=4000, step=100, value=2900, description='Wavenumber (cm^-1)'),
                 T=widgets.FloatSlider(min=200, max=500, step=10, value=298, description='T (K)'));

### Visualizing Quantum Tunneling

Tunneling occurs when a particle encounters a barrier higher than its energy ($E < V_0$) but still has a non-zero probability of being found on the other side. This is due to the wave nature of matter.

In [ ]:
def plot_tunneling(E_ratio, width):
    # E_ratio = E / V0
    x = np.linspace(-2, 4, 500)
    V0 = 1.0
    E = E_ratio * V0
    
    # Potential
    V = np.zeros_like(x)
    barrier_mask = (x > 0) & (x < width)
    V[barrier_mask] = V0
    
    plt.figure(figsize=(8, 5))
    plt.plot(x, V, 'k-', linewidth=2, label='Potential Barrier')
    plt.axhline(E, color='g', linestyle='--', label='Energy E')
    
    # Wavefunction (Schematic)
    psi = np.zeros_like(x)
    
    # Region 1 (x < 0): Incident + Reflected
    # Heuristic visualization
    k1 = np.sqrt(E) if E > 0 else 0.1
    psi[x <= 0] = np.cos(10*k1*x[x<=0]) # Incoming
    
    # Region 2 (0 < x < width): Barrier
    if E < V0:
        kappa = np.sqrt(V0 - E)
        # Decay
        psi[barrier_mask] = np.exp(-5*kappa*x[barrier_mask]) * 0.8 # scaled
    else:
        k2 = np.sqrt(E - V0)
        psi[barrier_mask] = np.cos(10*k2*x[barrier_mask]) * 0.8
        
    # Region 3 (x > width): Transmitted
    # Amplitude depends on tunneling probability
    if E < V0:
        kappa = np.sqrt(V0 - E)
        trans_amp = np.exp(-5*kappa*width)
        psi[x >= width] = trans_amp * np.cos(10*k1*(x[x>=width]-width))
    else:
        psi[x >= width] = 0.8 * np.cos(10*k1*(x[x>=width]-width))
        
    # Shift psi up to E for visualization
    plt.plot(x, psi + E, 'r-', label='Wavefunction')
    
    plt.title(f'Quantum Tunneling (E/V0 = {E_ratio})')
    plt.xlabel('Position')
    plt.ylabel('Energy / Amplitude')
    plt.legend()
    plt.ylim(0, 2.0)
    plt.show()

widgets.interact(plot_tunneling, 
                 E_ratio=widgets.FloatSlider(min=0.1, max=1.5, step=0.1, value=0.5, description='E / V0'),
                 width=widgets.FloatSlider(min=0.2, max=2.0, step=0.1, value=1.0, description='Barrier Width'));

## 5. Kinetic Salt Effect

For reactions between ions in solution: $A^{z_A} + B^{z_B} \rightarrow C^\ddagger \rightarrow P$

**Debye-Hückel Theory predicts:**
$$ \log k = \log k^\circ + 2 A z_A z_B \sqrt{I} $$

-   **Like charges** ($z_A z_B > 0$): Rate **increases** with ionic strength $I$. Ions screen the repulsion.
-   **Opposite charges** ($z_A z_B < 0$): Rate **decreases** with ionic strength $I$. Ions screen the attraction.
-   **Neutral** ($z_A z_B = 0$): No effect.

In [ ]:
def plot_salt_effect(zA, zB):
    I = np.linspace(0, 0.1, 100)
    A = 0.509  # Debye-Huckel constant for water at 25 C
    
    log_k_rel = 2 * A * zA * zB * np.sqrt(I)
    
    plt.figure(figsize=(8, 5))
    plt.plot(np.sqrt(I), log_k_rel, label=f'z_A={zA}, z_B={zB}')
    plt.xlabel('sqrt(Ionic Strength)')
    plt.ylabel('log(k / k0)')
    plt.title('Primary Kinetic Salt Effect')
    plt.axhline(0, color='k', linestyle='-', linewidth=0.5)
    
    # Annotations
    if zA * zB > 0:
        plt.text(0.1, 0.1, "Like Charges (++)\nRate Increases", color='blue')
    elif zA * zB < 0:
        plt.text(0.1, -0.1, "Opposite Charges (+-)\nRate Decreases", color='red')
    else:
        plt.text(0.1, 0.05, "Neutral (0)\nNo Effect", color='black')
        
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(plot_salt_effect, 
                 zA=widgets.IntSlider(min=-3, max=3, step=1, value=1, description='Charge A'),
                 zB=widgets.IntSlider(min=-3, max=3, step=1, value=1, description='Charge B'));

## 6. Linear Free Energy Relationships (LFER)

How does changing the structure of a molecule affect its reactivity? The **Hammett Equation** relates reaction rates to equilibrium constants for substituted benzenes:

$$ \log \left( \frac{k}{k_H} \right) = \rho \sigma $$

-   **$\sigma$ (Substituent Constant)**: Measures the electronic effect of the substituent (positive = electron withdrawing, negative = electron donating).
-   **$\rho$ (Reaction Constant)**: Measures the sensitivity of the reaction to electronic effects.
    -   $\rho > 0$: Reaction is accelerated by electron withdrawal (negative charge buildup in transition state).
    -   $\rho < 0$: Reaction is accelerated by electron donation (positive charge buildup in transition state).

In [ ]:
class HammettPlotter:
    """Interactive Hammett Plot"""
    
    def __init__(self):
        # Common substituents and their sigma values (para)
        self.substituents = {
            'p-NH2': -0.66, 'p-OCH3': -0.27, 'p-CH3': -0.17,
            'H': 0.00, 'p-Cl': 0.23, 'p-Br': 0.23,
            'm-Cl': 0.37, 'p-CN': 0.66, 'p-NO2': 0.78
        }
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.rho_slider = widgets.FloatSlider(
            min=-3.0, max=3.0, step=0.1, value=1.0, 
            description='Reaction ρ:'
        )
        
    def plot(self, rho):
        sigmas = list(self.substituents.values())
        labels = list(self.substituents.keys())
        
        # Calculate log(k/kH)
        log_k_rel = [rho * s for s in sigmas]
        
        plt.figure(figsize=(8, 6))
        plt.plot(sigmas, log_k_rel, 'b-', alpha=0.3, label=f'Slope ρ = {rho}')
        plt.scatter(sigmas, log_k_rel, c='red', s=100, zorder=5)
        
        # Label points
        for i, label in enumerate(labels):
            plt.annotate(label, (sigmas[i], log_k_rel[i]), 
                         xytext=(5, 5), textcoords='offset points')
            
        plt.axhline(0, color='k', linestyle=':', alpha=0.5)
        plt.axvline(0, color='k', linestyle=':', alpha=0.5)
        
        plt.xlabel('Substituent Constant σ')
        plt.ylabel('log(k / kH)')
        plt.title('Hammett Plot: Linear Free Energy Relationship')
        plt.legend()
        plt.grid(True)
        
        # Interpretation text
        if rho > 0.5:
            mech = "Nucleophilic attack (negative charge develops)"
        elif rho < -0.5:
            mech = "Electrophilic attack (positive charge develops)"
        else:
            mech = "Little charge development (radical or concerted)"
            
        plt.text(0.05, 0.95, f"Mechanism Hint:\n{mech}", 
                 transform=plt.gca().transAxes, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        plt.show()
        
    def setup_display(self):
        widgets.interact(self.plot, rho=self.rho_slider)

print("\n⚗️ Hammett Plot Explorer:")
hammett = HammettPlotter()

## Summary

1.  **Eyring Equation**: Relates rate constant to the free energy of the transition state.
2.  **Activation Entropy**: Negative $\Delta^\ddagger S$ indicates an ordered transition state (slow reaction); positive $\Delta^\ddagger S$ indicates a disordered one.
3.  **KIE**: H/D substitution affects rates via ZPE differences; $k_H/k_D > 1$ is normal.
4.  **Salt Effect**: Ionic strength modulates rates of ionic reactions by screening charges.
5.  **Hammett Equation**: Correlates structure and reactivity via electronic effects ($\rho\sigma$).